# Project 2


## Instructions

In this project, you will solve a two-dimensional reservoir simulation in a heterogenuous reservoir with multple wells.  Essentially, all of the functionality needed to do this was already implemented in [Homework Assignment 17](https://github.com/PGE323M-Fall2017/assignment17).  We will use real data from the Nechelik reservoir that we have looked at several times throughout the semester.  

As the inputs to our reservoir simulator are getting quite long now, we will use a special input file which contains all of the information about the reservoir fluid and rock properties, boundary conditions, well locations, and numerical inputs.  The file is shown [here](inputs.yml), which is in a special markup format that Python can easily read called [YAML](http://www.yaml.org/start.html).  There is a Python library that can easily convert this file to a Python dictionary and the code to do so was already implemented in the `__init__()` statement of the `TwoDimReservoir()` class.

For this project, you should implement the class below `Project2()` which inherits from `TwoDimReservoir` (which inherits from `OneDimReservoir`).  You may need to import these two base classes from the last assignments you used them ([Homework Assignment 13](https://github.com/PGE323M-Fall2017/assignment13) and [Homework Assignment 17](https://github.com/PGE323M-Fall2017/assignment17) in most cases) by converting those Jupyter Notebooks to Python files and placing the Python files in this repository.

**Please read this carefully!**  There are a few very minor changes you will want to make to the assignment13 and assignment17 codes.

 1. In your [Homework Assignment 17](https://github.com/PGE323M-Fall2017/assignment17) file, add the following import statement
 
    ```python
    import yaml
    ```

 1. The Nechelik field data has multiple cells that have a 0 value for permeability and porosity, unless you implemented a check previously for computing the interblock transmissibilities in cases where both $\kappa_i$ and $\kappa_{i+1}$ are 0, you may get an error like
 
 ```python
 RuntimeWarning: invalid value encountered in double_scalars
 ```
 
 you should implement some check for this special case in your code and assign the interblock permeabilities a value of 0.
 
 1. In [Homework Assignment 13](https://github.com/PGE323M-Fall2017/assignment13), you likely used the `spsolve()` function for solving the sparse system.  `spsolve()` has a check for singular matrices that will fail because the $\mathbf{T}$ matrix is singular which is caused by the zero permeability regions.  Please replace `spsolve()` with the method `cg()`.
 
 1. Finally, implement some functionality to read the porosity and permeability information from a file.  You will notice in [input.yml](input.yml), that these values take the filenames [`Nechelik_perm.dat`](Nechelik_perm.dat) and [`Neckelik_poro.dat`](Nechelik_poro.dat).  These files have the permeability and porosity data, respectively, for each grid block.
 
 
If you follow steps 1-4 above (and these are very minor changes), you may not need to write any additional code for your simulation to work.  However, it might be a good idea to write a few plotting routines to produce some plots like this one

![img](images/contour.png)

to help you determine if your code is working correctly.

## Testing

There are no locally available tests for this project, but if your `TwoDimReservoir` class passed all tests from [Homework Assignment 17](https://github.com/PGE323M-Fall2017/assignment17) after you make the changes 1-4 above, you can be reasonably assured it will work correctly.  Tests will be run on Travis and you will recieve feedback on whether they are passing or not upon submission. You can continue to resubmit until the deadline.

I encourage you to come up with your own tests as well.  One thing you can do is to work the project in CMG, which I have recorded a tutorial for [here](https://youtu.be/0wFy36pjdX8).  The tutorial covers the exact set of inputs shown in the [inputs.yml](inputs.yml) file and are summarized below.  The pressures in each grid block from your code should agree with CMG within 0.1 psi.

As you know, the actual Nechelik field has an irregular geometry as shown in the figure, with maximum $d = 100$ ft, $h = 5753$ ft and maximum $L = 7060.5$ ft. There are $N = 1188$ values in the data files corresponding to $N_x$ = 54 and $N_y$ = 22 grids to be used in the reservoir.  The reservoir has constant properties $\mu = 1$ cp, $B_\alpha = 1$, $c_t = 1 \times 10^{-6}$ psi$^{-1}$ and an inital reservoir pressure of $p_{\mbox{initial}} = 3700$ psi.

The reservoir has the following wells

|**Well**|**Location**<br> (ft, ft)|**Well type** | **Operating conditions** <br> (ft$^3$/day or psi)|
|:-:|:-:|:-:|:-:|
|1| 5536, 3500| Constant BHP | 2000 |
|2| 5474, 4708| Constant BHP | 2000 |
|3| 3600, 4937| Constant BHP | 2000 |
|4| 2400, 3322| Constant BHP | 2000 |
|5| 2500, 4050| Constant rate injector | 1000 |

All wells have a radius of $r_w = 0.25$ ft and negligible skin factor.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

#from ipywidgets import interact

from assignment17 import TwoDimReservoir

In [2]:
class Project2(TwoDimReservoir):      
            
    def __init__(self, inputs):
        
        super().__init__(inputs)
        
        self.p_plot = [self.get_solution()]
        
        
    def plot_pressure_field(self, i):
        
        #Reassignment for convenience, not a deep-copy
        dx = self.delta_x
        dy = self.delta_y

        #Compute grid centers
        grid_centers_x = np.cumsum(dx, axis=1) - dx[:,0,None] / 2.0
        grid_centers_y = np.cumsum(dy, axis=0) - dy[None, 0,:] / 2.0
        
        x_dim, y_dim = grid_centers_x.shape
        
        self.p_plot[i][self.permeability <= 1e-10] = np.nan
        
        z_value = self.p_plot[i].reshape(x_dim, y_dim)
        
        plt.contourf(grid_centers_x, grid_centers_y[::-1], z_value, levels=np.linspace(2000,3700, num=20))
        plt.colorbar()
        plt.show()
        
        return
    
    def plot_final_pressure_field(self):
        return self.plot_pressure_field(-1)

In [3]:
#pr = Project2('inputs.yml')
#pr.solve()

#@interact(i=(0,len(pr.p_plot)-1),continuous_update=False)
#def iplot(i=1):
#    return pr.plot_pressure_field(i)

interactive(children=(IntSlider(value=1, description='i', max=200), Output()), _dom_classes=('widget-interact',))